In [ ]:
import numpy as np
import mpmath

In [ ]:
from mpmath import mp

In [ ]:
mp.dps = 50 
_0 = mp.mpf(0)
_1 = mp.mpf(1)
_2 = mp.mpf(2)
_3 = mp.mpf(3)
_4 = mp.mpf(4)
_5 = mp.mpf(5)
_6 = mp.mpf(6)
_7 = mp.mpf(7)
_8 = mp.mpf(8)

In [ ]:
def det_sym(a):
    return a[0] * a[3] * a[5] + _2 * a[1] * a[2] * a[4] - a[0] * a[4] * a[4] - a[3] * a[2] * a[2] - a[5] * a[1] * a[1]

In [ ]:
def xT_adjA_x(x, a):
    return (x[0] * x[0] * (a[3] * a[5] - a[4] * a[4])
            + x[1] * x[1] * (a[0] * a[5] - a[2] * a[2])
            + x[2] * x[2] * (a[0] * a[3] - a[1] * a[1])
            + _2 * (x[0] * x[1] * (a[2] * a[4] - a[1] * a[5])
                    + x[0] * x[2] * (a[1] * a[4] - a[2] * a[3])
                    + x[1] * x[2] * (a[1] * a[2] - a[0] * a[4])))

In [ ]:
def rT_adjQ_r_as_poly(r, q1, q2, q3):
    a_zero = xT_adjA_x(r, q1)
    a_one = xT_adjA_x(r, q2)
    a_minus_one = xT_adjA_x(r, q3)
    return np.array([a_zero,
                     (a_one - a_minus_one)/_2,
                     (a_one + a_minus_one)/_2 - a_zero,])

In [ ]:
def detQ_as_poly(q1, q2, q3, q4):
    b_zero = det_sym(q1)
    b_one = det_sym(q2)
    b_minus_one = det_sym(q3)
    b_one_half = det_sym(q4)
    return np.array([b_zero,
                     (_8 * b_one_half - _6 * b_zero - _3/_2 * b_one - b_minus_one/_2) / _3,
                     (b_one + b_minus_one)/_2 - b_zero,                     
                     (-_8 * b_one_half + _6 * b_zero + _3 * b_one - b_minus_one) / _3])

In [ ]:
def spheroid(a, c, n):
    a2 = a * a
    c2_minus_a2 = c * c - a2
    nx = n[0]
    ny = n[1]
    nz = n[2]
    return np.array([nx * nx * c2_minus_a2 + a2,
                     nx * ny * c2_minus_a2,
                     nx * nz * c2_minus_a2,
                     ny * ny * c2_minus_a2 + a2,
                     ny * nz * c2_minus_a2,
                     nz * nz * c2_minus_a2 + a2])

In [ ]:
def contact_function(r12, q1, q2):
    q3 = _2*q1-q2
    q4 = (q1+q2)/_2
    a = rT_adjQ_r_as_poly(r12, q1, q2, q3)
    b = detQ_as_poly(q1, q2, q3, q4)
    
    c0 = a[0] * b[0]
    c1 = _2 * (a[1] - a[0]) * b[0]
    c2 = -a[0] * (b[1] + b[2]) + _3 * b[0] * (a[2] - a[1]) + a[1] * b[1]
    c3 = _2 * (b[1] * (a[2] - a[1]) - a[0] * b[3]) - _4 * a[2] * b[0]
    c4 = (a[0] - a[1]) * b[3] + (a[2] - a[1]) * b[2] - _3 * a[2] * b[1]
    c5 = -_2 * a[2] * b[2]
    c6 = -a[2] * b[3]
    xl = _0
    yl = c0
    xr = _1
    yr = c0 + c1 + c2 + c3 + c4 + c5 + c6
    while (mp.absmax(xr - xl) > 1e-15*(xl+xr)):
        x = (xl + xr)/_2
        y = c0 + x * (c1 + x * (c2 + x * (c3 + x * (c4 + x * (c5 + x * c6)))))
        if y == _0:
            break
        elif yl * y < _0:
            xr = x
            yr = y
        else:
            xl = x
            yl = y
    y = x * (_1 - x) * (a[0] + x * (a[1] + x * a[2])) / (b[0] + x * (b[1] + x * (b[2] + x * b[3])))
    return np.array([y, x])

In [ ]:
golden_ratio = (1.+mp.sqrt(_5))/_2

In [ ]:
norm = mp.sqrt(_1+golden_ratio**2)
u_abs = _1/norm
v_abs = golden_ratio/norm

In [ ]:
r = np.array([_0, u_abs, v_abs])
n1 = np.array([_0, u_abs, v_abs])
n2 = np.array([_0, u_abs, v_abs])
a1 = c1 = mp.mpf(1999)/mp.mpf(1000)
a2 = c2 = mp.mpf(199)/mp.mpf(10000)

In [ ]:
q1 = spheroid(a1, c1, n1)
q2 = spheroid(a2, c2, n2)

In [ ]:
mu2, lambda_ = contact_function(r, q1, q2)
print('mu2 = {}'.format(mu2))
print('lambda_ = {}'.format(lambda_))

In [ ]:
r_norm = np.linalg.norm(r)
r_norm

In [ ]:
mu2_expected = (r_norm/(a1+a2))**2
mu2_expected